In [3]:
import operator
import random

import numpy

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, 
    smin=None, smax=None, best=None)

def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))

toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", benchmarks.h1)

def main():
    pop = toolbox.population(n=5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
    
    return pop, logbook, best

if __name__ == "__main__":
    pop, logbook, best = main()
    print pop
    print logbook
    print best

gen	evals	avg      	std      	min      	max     
0  	5    	0.0799732	0.0302833	0.0462431	0.118372
1  	5    	0.0777365	0.0188239	0.0438808	0.0981052
2  	5    	0.0673366	0.035367 	0.0101285	0.10965  
3  	5    	0.0688613	0.0113935	0.0525877	0.083427 
4  	5    	0.0740676	0.0201179	0.0534997	0.107324 
5  	5    	0.053966 	0.0311396	0.0127452	0.10682  
6  	5    	0.0543806	0.0475293	0.0153085	0.142226 
7  	5    	0.0718694	0.0415586	0.0168437	0.133014 
8  	5    	0.0686081	0.0533549	0.00434806	0.130196 
9  	5    	0.0926738	0.0193912	0.0638993 	0.115571 
10 	5    	0.0669458	0.0379355	0.0171468 	0.130956 
11 	5    	0.084152 	0.0628493	0.00920317	0.178809 
12 	5    	0.105824 	0.0597826	0.0283402 	0.174179 
13 	5    	0.12452  	0.0825697	0.0224057 	0.214783 
14 	5    	0.259377 	0.151306 	0.104118  	0.496583 
15 	5    	0.453102 	0.503038 	0.0886161 	1.43063  
16 	5    	0.413558 	0.414071 	0.123332  	1.23628  
17 	5    	0.271258 	0.153731 	0.100768  	0.533056 
18 	5    	0.201822 	0.0924602	0.0739133 	0

302	5    	0.334762 	0.0876192	0.173032   	0.420531 
303	5    	0.625239 	0.385312 	0.266544   	1.25841  
304	5    	0.205951 	0.0741148	0.11835    	0.335133 
305	5    	0.33094  	0.0776014	0.201186   	0.414492 
306	5    	0.294915 	0.0848266	0.19481    	0.426165 
307	5    	0.508979 	0.477334 	0.188569   	1.44734  
308	5    	0.56297  	0.244381 	0.317681   	0.991436 
309	5    	0.272363 	0.0552753	0.211225   	0.359948 
310	5    	0.666602 	0.372536 	0.270606   	1.36389  
311	5    	0.551822 	0.358089 	0.107359   	1.02438  
312	5    	0.498176 	0.318193 	0.174432   	0.991484 
313	5    	0.501666 	0.283341 	0.188463   	0.996339 
314	5    	0.399944 	0.367386 	0.106707   	1.09382  
315	5    	0.44554  	0.217017 	0.0734748  	0.752746 
316	5    	0.533687 	0.542695 	0.0891887  	1.52259  
317	5    	0.436841 	0.173476 	0.220653   	0.737352 
318	5    	0.482779 	0.444506 	0.115117   	1.33023  
319	5    	0.456562 	0.354265 	0.0278126  	0.948605 
320	5    	0.368572 	0.430058 	0.00415084 	1.2115   
321	5    	0.